# Data Optimiser: Exploratory Data Analysis (EDA)

## Project Overview
Following the data integrity check, this notebook performs comprehensive exploratory data analysis to understand the job posting landscape for data roles.

### Key Questions to Explore:
1. What is the distribution of data roles in the market?
2. How do salaries vary across roles and experience levels?
3. Which locations have the highest demand for data professionals?
4. What are the most in-demand skills for each role?
5. How do company sizes affect job requirements and compensation?

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Load and clean the dataset
df = pd.read_csv('job_postings_dataset.csv')

# Basic data cleaning
# Standardize date format
df['posting_date_clean'] = pd.to_datetime(df['posting_date'])

# Remove rows with missing critical information
df_clean = df.dropna(subset=['job_title', 'company', 'location']).copy()

print(f"Original dataset: {len(df)} records")
print(f"Cleaned dataset: {len(df_clean)} records")
print(f"Records removed: {len(df) - len(df_clean)}")

## 1. Job Market Overview

In [ ]:
# Job title distribution
job_distribution = df_clean['job_title'].value_counts()
print("=== JOB MARKET OVERVIEW ===")
print("Job Title Distribution:")
for job, count in job_distribution.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{job}: {count} ({percentage:.1f}%)")

# Visualize job distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
job_distribution.plot(kind='bar', ax=ax1, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax1.set_title('Job Postings by Role', fontsize=14, fontweight='bold')
ax1.set_xlabel('Job Title')
ax1.set_ylabel('Number of Postings')
ax1.tick_params(axis='x', rotation=45)

# Pie chart
ax2.pie(job_distribution.values, labels=job_distribution.index, autopct='%1.1f%%',
        colors=['#FF6B6B', '#4ECDC4', '#45B7D1'], startangle=90)
ax2.set_title('Job Market Share by Role', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 2. Salary Analysis

In [ ]:
# Salary analysis by job title
print("=== SALARY ANALYSIS ===")

# Remove outliers for better visualization
salary_data = df_clean[df_clean['salary'].notna()]
Q1 = salary_data['salary'].quantile(0.25)
Q3 = salary_data['salary'].quantile(0.75)
IQR = Q3 - Q1
salary_clean = salary_data[(salary_data['salary'] >= Q1 - 1.5*IQR) & 
                          (salary_data['salary'] <= Q3 + 1.5*IQR)]

# Salary statistics by job title
salary_stats = salary_clean.groupby('job_title')['salary'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(0)

print("Salary Statistics by Job Title:")
print(salary_stats)

# Visualize salary distributions
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Box plot by job title
sns.boxplot(data=salary_clean, x='job_title', y='salary', ax=axes[0,0])
axes[0,0].set_title('Salary Distribution by Job Title', fontweight='bold')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Box plot by experience level
sns.boxplot(data=salary_clean, x='experience_level', y='salary', ax=axes[0,1])
axes[0,1].set_title('Salary Distribution by Experience Level', fontweight='bold')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Histogram of all salaries
salary_clean['salary'].hist(bins=30, ax=axes[1,0], alpha=0.7, color='skyblue')
axes[1,0].set_title('Overall Salary Distribution', fontweight='bold')
axes[1,0].set_xlabel('Salary ($)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Average salary by job title
avg_salary = salary_clean.groupby('job_title')['salary'].mean().sort_values(ascending=True)
avg_salary.plot(kind='barh', ax=axes[1,1], color='lightcoral')
axes[1,1].set_title('Average Salary by Job Title', fontweight='bold')
axes[1,1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.tight_layout()
plt.show()

## 3. Geographic Analysis

In [ ]:
# Geographic distribution analysis
print("=== GEOGRAPHIC ANALYSIS ===")

# Top locations by job count
location_counts = df_clean['location'].value_counts().head(10)
print("Top 10 Locations by Job Count:")
for location, count in location_counts.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{location}: {count} ({percentage:.1f}%)")

# Average salary by location (top 10 locations)
top_locations = location_counts.index[:10]
salary_by_location = salary_clean[salary_clean['location'].isin(top_locations)].groupby('location')['salary'].mean().sort_values(ascending=False)

print("\nAverage Salary by Top Locations:")
for location, avg_salary in salary_by_location.items():
    print(f"{location}: ${avg_salary:,.0f}")

# Visualize geographic data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

# Job count by location
location_counts.plot(kind='barh', ax=ax1, color='steelblue')
ax1.set_title('Top 10 Locations by Job Count', fontweight='bold')
ax1.set_xlabel('Number of Job Postings')

# Average salary by location
salary_by_location.plot(kind='barh', ax=ax2, color='orange')
ax2.set_title('Average Salary by Location', fontweight='bold')
ax2.set_xlabel('Average Salary ($)')
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.tight_layout()
plt.show()

# Job distribution by location and role
location_role_crosstab = pd.crosstab(df_clean['location'], df_clean['job_title'])
top_10_locations = df_clean['location'].value_counts().head(10).index
location_role_top = location_role_crosstab.loc[top_10_locations]

plt.figure(figsize=(12, 8))
location_role_top.plot(kind='bar', stacked=True, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
plt.title('Job Distribution by Location and Role (Top 10 Locations)', fontweight='bold')
plt.xlabel('Location')
plt.ylabel('Number of Job Postings')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Job Title')
plt.tight_layout()
plt.show()

## 4. Skills Demand Analysis

In [ ]:
# Skills analysis
print("=== SKILLS DEMAND ANALYSIS ===")

# Function to extract skills for each role
def get_skills_by_role(df, role):
    role_data = df[df['job_title'] == role]
    all_skills = []
    for skills_str in role_data['required_skills'].dropna():
        skills_list = [skill.strip() for skill in skills_str.split(',')]
        all_skills.extend(skills_list)
    return Counter(all_skills)

# Get top skills for each role
roles = df_clean['job_title'].unique()
role_skills = {}

for role in roles:
    skills_counter = get_skills_by_role(df_clean, role)
    role_skills[role] = skills_counter.most_common(10)
    
    print(f"\nTop 10 Skills for {role}:")
    for skill, count in skills_counter.most_common(10):
        percentage = (count / len(df_clean[df_clean['job_title'] == role])) * 100
        print(f"  {skill}: {count} ({percentage:.1f}%)")

# Visualize skills by role
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, role in enumerate(roles):
    skills_data = dict(role_skills[role])
    skills_df = pd.DataFrame(list(skills_data.items()), columns=['Skill', 'Count'])
    
    skills_df.plot(x='Skill', y='Count', kind='bar', ax=axes[i], 
                   color=['#FF6B6B', '#4ECDC4', '#45B7D1'][i], legend=False)
    axes[i].set_title(f'Top Skills for {role}', fontweight='bold')
    axes[i].set_xlabel('Skills')
    axes[i].set_ylabel('Frequency')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Overall most demanded skills
all_skills = []
for skills_str in df_clean['required_skills'].dropna():
    skills_list = [skill.strip() for skill in skills_str.split(',')]
    all_skills.extend(skills_list)

overall_skills = Counter(all_skills).most_common(15)

print("\n=== OVERALL TOP 15 MOST DEMANDED SKILLS ===")
for skill, count in overall_skills:
    percentage = (count / len(df_clean)) * 100
    print(f"{skill}: {count} ({percentage:.1f}%)")

# Create overall skills chart
overall_skills_df = pd.DataFrame(overall_skills, columns=['Skill', 'Count'])
plt.figure(figsize=(12, 8))
plt.barh(overall_skills_df['Skill'], overall_skills_df['Count'], color='lightgreen')
plt.title('Top 15 Most Demanded Skills (All Roles)', fontweight='bold', fontsize=16)
plt.xlabel('Frequency')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 5. Company and Experience Analysis

In [ ]:
# Company size analysis
print("=== COMPANY SIZE ANALYSIS ===")

# Distribution by company size
company_size_dist = df_clean['company_size'].value_counts()
print("Job Distribution by Company Size:")
for size, count in company_size_dist.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{size}: {count} ({percentage:.1f}%)")

# Average salary by company size
salary_by_company_size = salary_clean.groupby('company_size')['salary'].agg(['mean', 'median', 'count']).round(0)
print("\nSalary Statistics by Company Size:")
print(salary_by_company_size)

# Experience level analysis
print("\n=== EXPERIENCE LEVEL ANALYSIS ===")
exp_level_dist = df_clean['experience_level'].value_counts()
print("Job Distribution by Experience Level:")
for level, count in exp_level_dist.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{level}: {count} ({percentage:.1f}%)")

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Company size distribution
company_size_dist.plot(kind='pie', ax=axes[0,0], autopct='%1.1f%%')
axes[0,0].set_title('Job Distribution by Company Size', fontweight='bold')
axes[0,0].set_ylabel('')

# Experience level distribution
exp_level_dist.plot(kind='pie', ax=axes[0,1], autopct='%1.1f%%')
axes[0,1].set_title('Job Distribution by Experience Level', fontweight='bold')
axes[0,1].set_ylabel('')

# Salary by company size
salary_by_company_size['mean'].plot(kind='bar', ax=axes[1,0], color='coral')
axes[1,0].set_title('Average Salary by Company Size', fontweight='bold')
axes[1,0].set_ylabel('Average Salary ($)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Work arrangement distribution
work_arrangement_dist = df_clean['work_arrangement'].value_counts()
work_arrangement_dist.plot(kind='bar', ax=axes[1,1], color='lightblue')
axes[1,1].set_title('Job Distribution by Work Arrangement', fontweight='bold')
axes[1,1].set_ylabel('Number of Jobs')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Correlation Analysis

In [ ]:
# Create correlation analysis
print("=== CORRELATION ANALYSIS ===")

# Create numerical encodings for categorical variables
analysis_df = salary_clean.copy()

# Encode job titles
job_title_map = {'Data Analyst': 1, 'Data Scientist': 2, 'Data Engineer': 3}
analysis_df['job_title_encoded'] = analysis_df['job_title'].map(job_title_map)

# Encode experience levels
exp_level_map = {
    'Entry Level (0-2 years)': 1,
    'Mid Level (3-5 years)': 2, 
    'Senior Level (6-10 years)': 3,
    'Lead Level (10+ years)': 4
}
analysis_df['experience_encoded'] = analysis_df['experience_level'].map(exp_level_map)

# Encode company sizes
company_size_map = {
    'Startup (1-50)': 1,
    'Small (51-200)': 2,
    'Medium (201-1000)': 3,
    'Large (1001+)': 4
}
analysis_df['company_size_encoded'] = analysis_df['company_size'].map(company_size_map)

# Calculate number of skills
analysis_df['num_skills'] = analysis_df['required_skills'].apply(
    lambda x: len(x.split(', ')) if pd.notna(x) else 0
)

# Select numerical columns for correlation
correlation_columns = ['salary', 'job_title_encoded', 'experience_encoded', 
                      'company_size_encoded', 'num_skills']
correlation_matrix = analysis_df[correlation_columns].corr()

print("Correlation Matrix:")
print(correlation_matrix.round(3))

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.3f', cbar_kws={"shrink": .8})
plt.title('Correlation Matrix: Job Characteristics vs Salary', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

# Key insights from correlations
print("\n=== KEY CORRELATION INSIGHTS ===")
salary_correlations = correlation_matrix['salary'].sort_values(ascending=False)
for var, corr in salary_correlations.items():
    if var != 'salary':
        strength = "Strong" if abs(corr) > 0.5 else "Moderate" if abs(corr) > 0.3 else "Weak"
        direction = "positive" if corr > 0 else "negative"
        print(f"{var}: {corr:.3f} ({strength} {direction} correlation with salary)")

## 7. Time-Based Analysis

In [ ]:
# Time-based analysis
print("=== TIME-BASED ANALYSIS ===")

# Job postings by month
df_clean['posting_month'] = df_clean['posting_date_clean'].dt.to_period('M')
monthly_postings = df_clean.groupby('posting_month').size()

print("Job Postings by Month:")
for month, count in monthly_postings.items():
    print(f"{month}: {count} jobs")

# Job postings by role over time
monthly_by_role = df_clean.groupby(['posting_month', 'job_title']).size().unstack(fill_value=0)

# Visualize time trends
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Total monthly postings
monthly_postings.plot(kind='line', marker='o', ax=ax1, linewidth=2, markersize=6)
ax1.set_title('Job Postings Over Time (Total)', fontweight='bold')
ax1.set_xlabel('Month')
ax1.set_ylabel('Number of Job Postings')
ax1.grid(True, alpha=0.3)

# Monthly postings by role
monthly_by_role.plot(kind='line', marker='o', ax=ax2, linewidth=2, markersize=6)
ax2.set_title('Job Postings Over Time by Role', fontweight='bold')
ax2.set_xlabel('Month')
ax2.set_ylabel('Number of Job Postings')
ax2.legend(title='Job Title')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Seasonal patterns
df_clean['posting_quarter'] = df_clean['posting_date_clean'].dt.quarter
quarterly_postings = df_clean.groupby('posting_quarter').size()

print("\nJob Postings by Quarter:")
for quarter, count in quarterly_postings.items():
    print(f"Q{quarter}: {count} jobs")

plt.figure(figsize=(10, 6))
quarterly_postings.plot(kind='bar', color='skyblue')
plt.title('Job Postings by Quarter', fontweight='bold')
plt.xlabel('Quarter')
plt.ylabel('Number of Job Postings')
plt.xticks([0, 1, 2, 3], ['Q1', 'Q2', 'Q3', 'Q4'], rotation=0)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. EDA Summary and Key Insights

In [ ]:
# Generate comprehensive EDA summary
print("=== EXPLORATORY DATA ANALYSIS SUMMARY ===")
print("="*60)

# Market overview insights
print("🏢 MARKET OVERVIEW INSIGHTS")
job_dist = df_clean['job_title'].value_counts(normalize=True) * 100
dominant_role = job_dist.index[0]
print(f"   • {dominant_role} roles dominate the market ({job_dist.iloc[0]:.1f}% of postings)")
print(f"   • Total job postings analyzed: {len(df_clean):,}")
print(f"   • Geographic spread: {df_clean['location'].nunique()} unique locations")

# Salary insights
print(f"\n💰 SALARY INSIGHTS")
salary_by_role = salary_clean.groupby('job_title')['salary'].mean()
highest_paid_role = salary_by_role.idxmax()
lowest_paid_role = salary_by_role.idxmin()
print(f"   • Highest average salary: {highest_paid_role} (${salary_by_role.max():,.0f})")
print(f"   • Lowest average salary: {lowest_paid_role} (${salary_by_role.min():,.0f})")
print(f"   • Overall salary range: ${salary_clean['salary'].min():,.0f} - ${salary_clean['salary'].max():,.0f}")
print(f"   • Median salary across all roles: ${salary_clean['salary'].median():,.0f}")

# Skills insights
print(f"\n🔧 SKILLS INSIGHTS")
all_skills_counter = Counter(all_skills)
top_skill = all_skills_counter.most_common(1)[0]
print(f"   • Most in-demand skill: {top_skill[0]} (required in {top_skill[1]} jobs)")
avg_skills = df_clean['required_skills'].apply(lambda x: len(x.split(', ')) if pd.notna(x) else 0).mean()
print(f"   • Average skills per job: {avg_skills:.1f}")
print(f"   • Total unique skills identified: {len(all_skills_counter)}")

# Geographic insights
print(f"\n🌍 GEOGRAPHIC INSIGHTS")
top_location = df_clean['location'].value_counts().index[0]
top_location_count = df_clean['location'].value_counts().iloc[0]
top_location_pct = (top_location_count / len(df_clean)) * 100
print(f"   • Top job market: {top_location} ({top_location_count} jobs, {top_location_pct:.1f}%)")
if len(salary_by_location) > 0:
    highest_salary_location = salary_by_location.idxmax()
    print(f"   • Highest paying location: {highest_salary_location} (${salary_by_location.max():,.0f} avg)")

# Company insights
print(f"\n🏭 COMPANY INSIGHTS")
company_dist = df_clean['company_size'].value_counts(normalize=True) * 100
dominant_company_size = company_dist.index[0]
print(f"   • Most common company size: {dominant_company_size} ({company_dist.iloc[0]:.1f}% of jobs)")
work_dist = df_clean['work_arrangement'].value_counts(normalize=True) * 100
dominant_work = work_dist.index[0]
print(f"   • Most common work arrangement: {dominant_work} ({work_dist.iloc[0]:.1f}% of jobs)")

# Experience insights
print(f"\n📊 EXPERIENCE INSIGHTS")
exp_dist = df_clean['experience_level'].value_counts(normalize=True) * 100
dominant_exp = exp_dist.index[0]
print(f"   • Most common experience requirement: {dominant_exp} ({exp_dist.iloc[0]:.1f}% of jobs)")

# Correlation insights
print(f"\n🔗 KEY CORRELATIONS")
if 'experience_encoded' in correlation_matrix.columns:
    exp_salary_corr = correlation_matrix.loc['salary', 'experience_encoded']
    print(f"   • Experience level vs Salary correlation: {exp_salary_corr:.3f}")
if 'num_skills' in correlation_matrix.columns:
    skills_salary_corr = correlation_matrix.loc['salary', 'num_skills']
    print(f"   • Number of skills vs Salary correlation: {skills_salary_corr:.3f}")

print(f"\n✅ EDA COMPLETE - Ready for Business Question Investigation")
print("="*60)

## 9. Power BI Translation Notes

### For Power BI Implementation:

**Key Visualizations to Create in Power BI:**
1. **Job Market Overview Dashboard**
   - Pie chart for job title distribution
   - Bar chart for geographic distribution
   - Card visuals for key metrics (total jobs, avg salary, etc.)

2. **Salary Analysis Dashboard**
   - Box plots using custom visuals for salary distributions
   - Scatter plot for salary vs experience correlation
   - Table with salary statistics by role

3. **Skills Demand Dashboard**
   - Horizontal bar charts for top skills by role
   - Word cloud visual for overall skills
   - Matrix visual showing skills by job title

**DAX Measures to Create:**
- `Average Salary = AVERAGE(JobData[salary])`
- `Job Count = COUNTROWS(JobData)`
- `Skills Count = LEN(JobData[required_skills]) - LEN(SUBSTITUTE(JobData[required_skills], ",", "")) + 1`
- `Salary by Experience = CALCULATE(AVERAGE(JobData[salary]), FILTER(JobData, JobData[experience_level] = "Selected Level"))`

**Power Query Transformations:**
- Split skills column by delimiter
- Create calculated columns for experience level ranking
- Extract state/city from location field
- Create salary ranges/buckets for better visualization

**Next Steps:**
Proceed to Business Question Investigation to focus on specific insights for Data Optimiser's recruitment strategy.